# Instationary Navier-Stokes

We consider the approximation of the full non-linear Navier-Stokes equations. As a first step we use an $H(\operatorname{div})$ conforming approach and consider an IMEX operator splitting, see [NGSolve docu - Navier Stokes](https://docu.ngsolve.org/latest/i-tutorials/unit-3.2-navierstokes/navierstokes.html).


As example we want to solve a flow through a Tesla valve ([tesla.py](tesla.py)), see US patent 1329559 "Valvular Conduit".

The original drawing by N. Tesla in his patent:

<img src="figures/tesla_patent.png" alt="tesla" width="650" align="center"/>

In [ ]:
from ngsolve import *
from netgen.occ import *
from ngsolve.webgui import Draw
import ipywidgets as widgets

In [ ]:
def GetValve(N, dim, R = 1.4, alpha = 45, Winlet = 1, beta = 190, L1 = 11, L2 = 6, Linlet = 5, closevalve = False):
    alphar = alpha * pi/180
    W = 1/cos(alphar/2)
    facW = 1
    Winlet2 =  Winlet
        
    wp = WorkPlane()
    p1 = 0
    p2 = 0

    wp.MoveTo(p1,p1) 
    r1 = Rectangle(L1,W).Face()
    
    r2 = wp.MoveTo(p1,p2+W).Rotate(-90+alpha).Move(W).Rotate(90).Rectangle(L2,W).Face()
    
    wp.MoveTo(p1,p2+W).Move(L2)
    c1 = Face(wp.Arc(W + R, -beta).Line(L1).Rotate(-90).Line(W).Rotate(-90).Line(L1).Arc(R,beta).Close().Wire())
    wp.MoveTo(0,W).Direction(1,0)
    cutplane = Rectangle(2*L1,4*L1).Face()
    

    v1 = r1 + r2  + (cutplane*c1) 
    
    ll = L1 + L1 * cos(alphar) - cos(alphar) * W
    hh = L1 * sin(alphar) - (1-sin(alphar)) * W
    didi = sqrt((L1 + L1 * cos(alphar))**2 + (L1 * sin(alphar))**2 ) - 2*W * sin(alphar/2) 
    
    dd = gp_Dir(cos(alpha), sin(alpha),0)
    v2 = v1.Mirror(Axis((0,0,0), X)).Move((0,W,0)).Rotate(Axis((0,0,0), Z), alpha).Move((L1,0,0))

    onevalve = (v1 + v2.Reversed()).Move((0,-W/2,0)).Rotate(Axis((0,0,0), Z), -alpha/2)

    vv = onevalve
    
    for i in range(1,N):
        vv = onevalve.Move((didi * i ,0, 0)) + vv
        
    
    
    inlet = wp.MoveTo(-Linlet,-Winlet2/2).Rectangle(Linlet* facW + W * sin(alphar/2) ,Winlet2).Face()
    outlet = wp.MoveTo(didi*N + W/2 * sin(alphar/2)* facW,-Winlet2/2).Rectangle(Linlet ,Winlet2).Face()
    vv = inlet + vv + outlet
    
    if closevalve == True:
        c1_x = -Linlet
        c1_y = -Winlet2/2

        c2_x = didi*N + W/2 * sin(alphar/2)* facW + Linlet
        c2_y = -Winlet2/2
        
        wp.MoveTo(c1_x, c1_y).Direction(-1,0)
        R2 = 3
        LL = c2_x - c1_x
        # close = Face(wp.Arc(R2+Winlet2,180).Line(LL).Arc(R2+Winlet2,180).Rotate(90).Line(Winlet2).Rotate(90).Arc(R2,-180).Line(LL).Arc(R2,-180).Close().Wire())
        
        close = Face(wp.Arc(R2+Winlet2,-180).Line(LL).Arc(R2+Winlet2,-180).Rotate(-90).Line(Winlet2).Rotate(-90).Arc(R2,180).Line(LL).Arc(R2,180).Close().Wire()).Reversed()
        # close.face.name

    teslavalve = vv
    # vv = onevalve
    if dim == 3:
        teslavalve = vv.Extrude( Winlet*Z )
        teslavalve.faces.name="wall"
        
    
        if closevalve == False:
            teslavalve.faces.Min(X).name="inlet"
            teslavalve.faces.Max(X).name="outlet"
            teslavalve.faces.Min(X).Identify(teslavalve.faces.Max(X),"periodic", IdentificationType.PERIODIC)

    else:
        teslavalve.edges.name="wall"
        teslavalve.faces.name="valve"
        teslavalve.edges.Min(X).name="inlet"
        teslavalve.edges.Max(X).name="outlet"
        
        if closevalve == True:
            close.edges.name = "wall"
            teslavalve =  Glue([teslavalve, close])
    
        if closevalve == False:        
            teslavalve.edges.Min(X).Identify(teslavalve.edges.Max(X),"periodic", IdentificationType.PERIODIC)

    return teslavalve

In [ ]:
valve = GetValve(N = 3, dim = 2, R = 0.5, alpha = 25, Winlet = 1, beta = 180, L1 = 6.4, L2 = 7, Linlet = 5, closevalve = True)
mesh = Mesh(OCCGeometry(valve, dim=2).GenerateMesh(maxh = 0.5)).Curve(3)
Draw(mesh);

We consider wall boundary conditions everywhere, and as force and viscosity we choose

$$
(f,v) = r
\begin{pmatrix}
1 \\ 0
\end{pmatrix},
\qquad 
\text{and}
\qquad 
\nu = 0.005,
$$

where the factor $r \in \{-1,1\}$ will determine the flow direction (left to right, or right to left). Our goal is to evaluate the mass flux through the ```inlet``` boundary 

$$
flux(u_h) = \int_{\Gamma_{in}} u_h \cdot n,
$$

over a period of $T_{end} = 30$.

In [ ]:
nu = 0.005   # viscosity
tau = 0.002  # time step
tend = 50    # end time
order = 2    # fem order (velocity)

For this example we want to make use of the ```hodivfree``` flag for the ```HDiv``` space. In NGSolve the hierarchical basis functions are split into four groups


<img src="figures/sep_lo.png" alt="lodof" width="150px" align="center">
<img src="figures/sep_facedofs.png" alt="facedof" width="150px" align="center">
<img src="figures/sep_eldofsdiv.png" alt="divdof" width="150px" align="center">
<img src="figures/sep_eldofsdivfree.png" alt="divfreedof" width="150px" align="center">


- The first group corresponds to the lowest order Raviart-Thomas $\mathcal{RT}_0$ space whose basis functions are associated to facets. Further note that the divergence of these functions is element wise constant, $\operatorname{div}{\mathcal{RT}_0} = \mathbb{P}^0 = Q_h^0$, which corresponds to the lowest order pressure space.
- The second group are the remaining facet basis functions which are *all divergence free*. 
- The third group represents n-bubbles (functions with a vanishing normal trace) with a non-zero divergence. More precisely, they correspond to the remaining high order part of the pressures, $Q_h^0/Q_h^0 = \mathbb{P}^{k-1}/\mathbb{P}^0$.
- The last group are also n-bubbles, but are all divergence free.

Since we want our velocity solution $u_h$ to be divergence-free, we already know beforehand that we can exclude the third group of basis functions, ```hodivfree=True```.  Although this motivates to remove also the lowest order Raviart-Thomas functions (producing a divergence) this can not be done since they are non-local. Accordingly, we still need the lowest-order pressure as a Lagrange multiplier in our system.


*More information on the basis function in NGSolve can be found in the PhD thesis of Sabine Zaglmayr "High order finite element methods for electromagnetic field computation". For the $H(div)$-space we also refer to "P.L Lederer, C. Lehrenfeld, J. Schöberl, Hybrid discontinuous Galerkin methods with relaxed H(div)-conformity for incompressible flows, https://doi.org/10.1137/17M1138078"*


In [ ]:
VT =   HDiv(mesh,order=order, dirichlet="wall", hodivfree=True)
Vhat = TangentialFacetFESpace(mesh,order=order, dirichlet="wall")
Q = L2(mesh,order=0)

X = VT * Vhat * Q

uh, vh = X.TnT()
u, uhat, p = uh
v, vhat, q = vh

n = specialcf.normal(mesh.dim)
h = specialcf.mesh_size

def tang(u): # tangential part of a vector field
    return u - (u*n)*n

We want to use an IMEX splitting method (see [NGSolve docu - Navier Stokes](https://docu.ngsolve.org/latest/i-tutorials/unit-3.2-navierstokes/navierstokes.html)). Hence, let $(u,\hat u, p)^n$ be the coefficient vector of the discrete solution $u_h^n = u_h(t_n), \hat u_h^n = \hat u_h(t_n)$ and $p_h^n = p_h(t_n)$ for an aequidistant mesh $0 = t_0 < \ldots < t_N = T_{end}$. We want to solve the system

$$
\underbrace{\left(
\begin{pmatrix}
M & 0 & 0\\ 0 & 0 & 0 \\ 0 & 0 & 0
\end{pmatrix} + \tau K\right)}_{= M^*} 
\begin{pmatrix}
u \\ \hat u \\ p
\end{pmatrix}^{n+1} 
=
\begin{pmatrix}
u \\ \hat u \\ p
\end{pmatrix}^{n} 
+ \tau \left(f - K \begin{pmatrix}
u \\ \hat u \\ p
\end{pmatrix}^{n+1}  - C(u^n)u^n \right),
$$

where $M$ is the mass matrix on ```VT``` and $C(u^n)u^n$ is an explicit evaluation of the non-linear convection term (where we integrated by parts)

$$
[C(u^n)u^n]_i = \sum_T -\int_T u_h^n \otimes u_h^n : \nabla \phi_i + \int_{\partial T} u_h^n \cdot n (u_h^n)^{up} \cdot \phi_i,
$$

where $\phi_i$ is a basis function of the space ```VT```, and $(u_h^n)^{up}$ is the upwinded velocity.

In [ ]:
dS = dx(element_boundary = True)

stokes = (nu*InnerProduct(Grad(u), Grad(v)) - div(u)*q - div(v)*p - 1e-10/nu*p*q)*dx
stokes += nu * Grad(u)*n * tang(v-vhat) * dS
stokes += nu * Grad(v)*n * tang(u-uhat) * dS
stokes += nu * 10 * order**2/h * tang(v-vhat)*tang(u-uhat) * dS

K = BilinearForm(stokes, symmetric=True).Assemble()
mstar = BilinearForm(u*v*dx + tau*stokes, symmetric=True).Assemble()
inv = mstar.mat.Inverse(X.FreeDofs(), inverse = "sparsecholesky")

The right hand side is only assembled once and will me multiplied with the factor $r$ in the solving routine.

In [ ]:
f = LinearForm(CF((1,0)) * v * dx(definedon = mesh.Materials("valve"))).Assemble()   

In [ ]:
conv = BilinearForm(X, nonlinear_matrix_free_bdb=True)
conv += -InnerProduct(grad(v)*u, u) * dx(bonus_intorder=order)

un = u.Operator("normalcomponent")
uhat_t = uhat.Operator("tangentialcomponent")
vn = v.Operator("normalcomponent")
conv += IfPos(un, un * u * v, 2 * un * un * vn + un * ((2*uhat_t-u) * v) ) * dx(element_boundary = True, bonus_intorder=order)
conv.Assemble()
conv_operator = conv.mat

In order to calculate the mean value $\tilde u_h$ we set up the $L^2$-projection problem (for a given $u_h$)

$$
\sum_T \int_{\partial T} (\tilde u_h)_t (\hat v_h)_t = \sum_T \int_{\partial T} (u_h)_t (\hat v_h)_t
\quad 
\forall \hat v_h \in \hat V_h.
$$

Since the tangential component of functions in ```Vhat``` are single valued on facets (in contrast to $u_h$) we have on a common facet 

$$ 
2 (\tilde u_h)_t (\hat v_h)_t = \Big((u_h)_t + (u_h^{other})_t \Big)(\hat v_h)_t.
$$

In [ ]:
u1, v1 = VT.TnT()
uh1, vh1 = Vhat.TnT()

massfacet = BilinearForm(tang(uh1)*tang(vh1)*dx(element_boundary=True)).Assemble()
mixedmassfacet = BilinearForm(u1*tang(vh1)*dx(element_boundary=True)).Assemble()
invfacet = massfacet.mat.CreateBlockSmoother(Vhat.CreateSmoothingBlocks(blocktype="facet"))

eu, euhat, ep = X.embeddings
ru, ruhat, rp = X.restrictions
avg_op = eu@ru + euhat@invfacet@mixedmassfacet.mat@ru

In [ ]:
gfu = GridFunction(X)

def Solve(factor=1, callback = lambda: None):
    t = 0
    cnt = 0
    with TaskManager():
        while t < tend:
          res = factor*f.vec - conv_operator@avg_op * gfu.vec - K.mat * gfu.vec
          gfu.vec.data += tau * inv * res    
          
          t += tau
          cnt += 1
          callback (cnt, gfu)

In [ ]:
massflux = []
massflux_reverse = []
time = []
gfut = GridFunction(gfu.components[0].space, multidim=0)
gfut_reverse = GridFunction(gfu.components[0].space, multidim=0)


scene = Draw(gfu.components[0], mesh, min=0, max=4, autoscale=False, vectors = {"grid_size": 100})
tw = widgets.Text(value='t = 0')
display(tw)

def cbforward (cnt, gfu):
    if cnt % 100 == 0: 
        scene.Redraw()
        tw.value = "t = "+str(cnt*tau)
    if cnt % 500 == 0:
        gfut.AddMultiDimComponent(gfu.components[0].vec)
        massflux.append(abs(Integrate(gfu.components[0]*n, mesh.Boundaries("inlet"))))
        time.append(cnt*tau)

def cbreverse (cnt, gfu):
    if cnt % 100 == 0: 
        scene.Redraw()
        tw.value = "t = "+str(cnt*tau)
    if cnt % 500 == 0:
        gfut_reverse.AddMultiDimComponent(gfu.components[0].vec)
        massflux_reverse.append(abs(Integrate(gfu.components[0]*n, mesh.Boundaries("inlet"))))



gfu.vec[:] = 0
Solve (1, cbforward)

gfu.vec[:] = 0
Solve (-1, cbreverse)


In the following plots we can see the solutions at $t = T_{end}$. We observe that for the flow from left to right the solution is much more turbulent and the valve is "activated" since the fluid stops it self. On the other hand, for the flow from right to left, the solution is nearly laminar and much faster. This can also be observed in the  the maxs flux ploted over time below.

In [ ]:
Draw (gfut, mesh, interpolate_multidim=True, animate=True,
      min=0, max=4, autoscale=False, vectors = {"grid_size": 100});
Draw (gfut_reverse, mesh, interpolate_multidim=True, animate=True,
      min=0, max=6, autoscale=False, vectors = {"grid_size": 100});

In [ ]:
import matplotlib.pyplot as plt

plt.plot(time, massflux, "-", color = "orange", label = "left_to_right")
plt.plot(time, massflux_reverse, "-", color = "blue", label = "right_to_left")
plt.legend(loc = "lower right")
plt.show()
